# Thief Detector
## This task tests your Image Processing skills to build a motion detection algorithm that alarms you when you have an unwanted visitor in your home.

## Steps
- 1. Get the live video feed from your webcam
- 2. Fix a scene (the place you want to monitor) and store it as a reference background image
    - Store the first frame as the reference background frame
- 3. For every frame, check if there is any unwanted object inside the scene you are monitoring
    - Use **Background Subtraction** concept (**cv2.absdiff( )**)
        - Subtract the current frame from the reference background image(frame) to see the changes in the scene
        - If there is enormous amount of pixels distrubed in the subtraction result image
            - unwanted visitor (place is unsafe --> alarm the authorities)
        - If there is no enormous amount of pixels distrubed in the subtraction result image
            - no unwanted visitor (place is safe)
- 4. Output the text **"UNSAFE"** in **red** color on the top right of the frame when there is an intruder in the scene.
- 5. Save the live feed
- 6. Submit the (.ipynb) file

## Get live video feed from webcam [10 points]

In [2]:
import cv2
import time
from datetime import datetime

In [12]:
video = cv2.VideoCapture(0)  

## Read first frame, convert to Grayscale and store it as reference background image [10 points]

In [13]:
initial_state = None  

In [14]:
motion_track_list= [ None, None ]

In [15]:
motionTime = []

In [16]:
check, cur_frame = video.read()  

In [34]:
var_motion = 0  

In [18]:
gray_image = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)  

In [35]:
gray_frame = cv2.GaussianBlur(gray_image, (21, 21), 0) 

In [22]:
if initial_state is None:  
    initial_state = gray_frame  

## Compute Absolute Difference between Current and First frame [20 points]

In [26]:
differ_frame = cv2.absdiff(initial_state, gray_frame) 

## Apply threshold [5 points]

In [27]:
thresh_frame = cv2.threshold(differ_frame, 30, 255, cv2.THRESH_BINARY)[1]  

In [28]:
thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2) 

## Find contours [10 points]

In [29]:
cont,_ = cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  

## Check if contourArea is large and draw rectangle around the object, output "UNSAFE" text in red color [30 points]

In [30]:
for cur in cont:
    if cv2.contourArea(cur) < 10000:  
        continue  
        var_motion = 1  
        (cur_x, cur_y,cur_w, cur_h) = cv2.boundingRect(cur)  
# To create a rectangle of green color around the moving object  
        cv2.rectangle(cur_frame, (cur_x, cur_y), (cur_x + cur_w, cur_y + cur_h), (0, 255, 0), 3) 

## Display images [10 points]

In [12]:
cv2.imshow("Gray Frame: ", gray_frame)  
cv2.imshow("Difference Frame: ", differ_frame)   
cv2.imshow("Threshold Frame: ", thresh_frame)  
cv2.imshow("Colour Frame:", cur_frame)  
wait_key = cv2.waitKey(0)  

## Release objects [5 points]

In [10]:
video = cv2.VideoCapture(0) 
initial_state = None  
motion_track_list= [ None, None ]
motionTime = []
while True:
    check, cur_frame = video.read()  
    var_motion = 0  
    gray_image = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)  
    gray_frame = cv2.GaussianBlur(gray_image, (21, 21), 0)  
    if initial_state is None:
        initial_state = gray_frame 
        continue
    differ_frame = cv2.absdiff(initial_state, gray_frame)  
    thresh_frame = cv2.threshold(differ_frame, 30, 255, cv2.THRESH_BINARY)[1]  

    thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2)  

    cont,_ = cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    for cur in cont:
        if cv2.contourArea(cur) < 10000:
            continue  
        var_motion = 1  
        (cur_x, cur_y,cur_w, cur_h) = cv2.boundingRect(cur)  
        cv2.rectangle(cur_frame, (cur_x, cur_y), (cur_x + cur_w, cur_y + cur_h), (255, 255, 255), 3)
        cv2.putText(cur_frame, text ='UNSAFE', org=(40,50), fontFace =cv2.FONT_HERSHEY_TRIPLEX, fontScale =1, color=(255,255,255), thickness =1)
    motion_track_list.append(var_motion)  

    motion_track_list = motion_track_list[-2:]  

    if motion_track_list[-1] == 1 and motion_track_list[-2] == 0:
        motionTime.append(datetime.now())  

    if motion_track_list[-1] == 0 and motion_track_list[-2] == 1:
        motionTime.append(datetime.now())  
    cv2.imshow("Gray Frame: ", gray_frame)  

    cv2.imshow("Difference Frame: ", differ_frame)  
    
    cv2.imshow("Threshold Frame: ", thresh_frame)  
    
    cv2.imshow("Colour Frame:", cur_frame)  
    wait_key = cv2.waitKey(1)  

    if wait_key == ord('q'):  
        if var_motion == 1:  
            motionTime.append(datetime.now())  
        break 
video.release()  
cv2.destroyAllWindows()